# Import

In [26]:
import csv
import json
import logging
import os
import shutil
import sys
from typing import List
from glob import glob

from itertools import product
import boto3
import pandas as pd
from botocore.exceptions import ClientError

## Add configuration file

In [27]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")

In [28]:
from ALL import config 
from util import *
from MultilayerDict import *

In [29]:
s3_bucket_name =  "text-classification-nakayama-bucket"
object_name = "RawData/reuters/"
root_path = "/home/jovyan"

In [30]:
d = {"temporary": {"A": "doc", "B": "tes"}}

In [31]:
file_path = "../../temporary/B/test.csv"
dir_path = "../../temporary/"

In [32]:
f = pd.read_csv("../../temporary/B/test.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../../temporary/B/test.csv'

In [33]:
encoding="utf8"
newline=""
delimiter=","
quotechar="|"

In [37]:
def _one_dimensional_csv_to_dict(
    file_path, encoding="utf8", newline="", delimiter=",", quotechar="|"
):
    with open(file_path, mode="r", encoding=encoding, newline=newline) as _f:
        _reader = csv.reader(_f, delimiter=delimiter, quotechar=quotechar)
        _dict = dict(_reader)
    return _dict

In [38]:
def _one_dimensional_json_to_dict(
    file_path,
    encoding="utf8",
    newline="",
):
    with open(file_path, mode="r", encoding=encoding, newline=newline) as _f:
        _dict = json.load(_f)
    return _dict

In [39]:
def _make_test(
    file_data: pd.DataFrame,
    file_num: int = 2,
    file_depth: int = 2,
    extension: str = "csv",
):
    _file_nums = [str(_file_num) for _file_num in range(file_num)]

    for _file_name in product(_file_nums, repeat=file_depth):
        _file_path = f"{root_path_temporary}/test/{'/'.join(_file_name)}/test.{extension}"
        file_data.to_csv(make_filepath(_file_path), index=False)

In [40]:
test_df = pd.DataFrame([[1, 2,3 ], [2, 3, 4], [5, 6, 7]])

In [41]:
_make_test(test_df, file_depth=2)

In [46]:
field_names= ["a", "b", "c"]
index=["dim1", "dim2", "dim3", "dim4"]

In [58]:
names_keys = {"ind": index, "col": field_names}

In [59]:
def read_csv(
    file_path, names_keys, encoding="utf8", newline="", delimiter=",", quotechar="|"
):
    if len(names_keys) == 2:
        with open(file_path, mode="r", encoding=encoding, newline=newline) as _f:
            _dict = dict(
                zip(
                    list(names_keys.values())[0],
                    csv.DictReader(
                        _f,
                        delimiter=delimiter,
                        quotechar=quotechar,
                        fieldnames=list(names_keys.values())[1],
                    ),
                )
            )
        return MultilayerDict(names_keys, _dict)
    elif len(names_keys) == 1:
        with open(file_path, mode="r", encoding=encoding, newline=newline) as _f:
            _dict = csv.DictReader(
                        _f,
                        delimiter=delimiter,
                        quotechar=quotechar,
                        fieldnames=list(names_keys.values())[0],
                    )
        return MultilayerDict(names_keys, _dict)
    else:
        raise NotImplementedError("length of names_keys must be 1 or 2 ")

In [62]:
_two_dimensional_csv_to_dict("/home/jovyan/temporary/test/0/0/test.csv", names_keys)

In [ ]:
def read_dirs(dir_path, extension="csv", basis_depth="max"):
    # 読み込みディレクトリの中のファイルパスを探索し,ファイルパスからdictのkeyを生成
    _path_dirs = {}
    _abs_dir_path = os.path.abspath(dir_path)
    for _root, _, _file_names in os.walk(_abs_dir_path, followlinks=True):
        if _file_names:
            for _file_name in _file_names:
                _file_path = f"{_root}/{_file_name}"
                _file_name = os.path.splitext(os.path.basename(_file_name))[0]
                _path_dirs[_file_path] = (_file_path.replace(_abs_dir_path, "")).split(
                    "/"
                )[1:]
    # 最も深いパスを基準にMultilayerDictを生成する
    if basis_depth == "max":
        _depth = max(map(len, _path_dirs.values()))
    elif basis_depth == "min":
        _depth = min(map(len, _path_dirs.values()))
    else:
        raise NotImplementedError
    _names = range(_depth)
    _keys = [list(set(_)) for _ in zip(*_path_dirs.values())]
    _md = MultilayerDict(dict(zip(_names, _keys)))
    # ファイルの読み込み
    if extension == "csv":
        for _path, _dir_list in _path_dirs.items():
            md.update(_dir_list, _one_dimensional_csv_to_dict(_path))
    elif extension=="json":
        for _path, _dir_list in _path_dirs.items():
            md.update(_dir_list, _one_dimensional_csv_to_dict(_path))
    else:
        raise NotImplementedError
        
    return True

In [47]:
_path_dirs = {}
_abs_dir_path = os.path.abspath(dir_path)
for _root, _, _file_names in os.walk(_abs_dir_path, followlinks=True):
    if _file_names:
        for _file_name in _file_names:
            _file_path = f"{_root}/{_file_name}"
            _file_name = os.path.splitext(os.path.basename(_file_name))[0]
            _path_dirs[_file_path] = (_file_path.replace(_abs_dir_path, "")).split("/")[1:]

In [48]:
_depth = max(map(len, _path_dirs.values()))
_names = range(_depth)

In [49]:
_keys = [list(set(_)) for _ in zip(*_path_dirs.values())]

In [50]:
md = MultilayerDict(dict(zip(_names, _keys)))

In [51]:
md.dict

{'A': {'test2.csv': {}, 'test.csv': {}},
 'B': {'test2.csv': {}, 'test.csv': {}}}

In [52]:
_path

'/home/jovyan/temporary/B/test'

In [53]:
for _path, _dir_list in _path_dirs.items():
    md.update(_dir_list, one_dimensional_csv_to_dict(_path))

In [54]:
md.dict

{'A': {'test2.csv': {'a': ' 1', 'b': ' 2'},
  'test.csv': {'a': ' 1', 'b': ' 2'}},
 'B': {'test2.csv': {'a': ' 1', 'b': ' 2'},
  'test.csv': {'a': ' 1', 'b': ' 2'}}}